In [31]:
from pyspark.sql import SparkSession
from datetime import datetime, timezone

ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")

spark = SparkSession.builder \
    .appName(f"AAnalyses-{ts}") \
    .getOrCreate()

In [32]:
from pyspark.sql.functions import current_timestamp, to_utc_timestamp, col
from pyspark.sql.types import DecimalType

In [33]:
df_purchase_silver = spark.read.parquet("data_lake/silver/purchase")
df_purchase_extra_info_silver = spark.read.parquet("data_lake/silver/product_item")
df_product_item_silver = spark.read.parquet("data_lake/silver/purchase_extra_info")

In [34]:
df_purchase_silver.createOrReplaceTempView("purchase_silver")
df_purchase_extra_info_silver.createOrReplaceTempView("purchase_extra_info_silver")
df_product_item_silver.createOrReplaceTempView("product_item_silver")

In [35]:
spark.sql("SELECT * FROM purchase_silver ORDER BY purchase_id, transaction_datetime DESC").show()
spark.sql("SELECT * FROM purchase_extra_info_silver ORDER BY purchase_id, transaction_datetime DESC").show()
spark.sql("SELECT * FROM product_item_silver ORDER BY purchase_id, transaction_datetime DESC").show()

+--------------------+-----------+--------+------------+----------+------------+-----------+---------------+--------------------+---------------------+---------+----------------+
|transaction_datetime|purchase_id|buyer_id|prod_item_id|order_date|release_date|producer_id|invoiced_status|     line_created_at|bronze_ingestion_date|is_latest|transaction_date|
+--------------------+-----------+--------+------------+----------+------------+-----------+---------------+--------------------+---------------------+---------+----------------+
| 2023-07-15 09:00:00|         55|  160001|           5|2023-01-20|  2023-03-01|     852852|       Invoiced|2026-01-14 23:17:...| 2026-01-14 23:08:...|     true|      2023-07-15|
| 2023-02-05 10:00:00|         55|  160001|           5|2023-01-20|  2023-01-20|     852852|       Invoiced|2026-01-14 23:17:...| 2026-01-14 23:08:...|    false|      2023-02-05|
| 2023-01-20 22:00:00|         55|   15947|           5|2023-01-20|  2023-01-20|     852852|       Invoic

In [36]:
df_gvm_existing = spark.read.parquet("data_lake/gold/gvm")

In [37]:
# Incremental dos registros mais recentes de todas as tabelas no dia da carga. 
# "Nesse dia o resultado foi x. 
df_new_gvm = spark.sql("""
    SELECT 
        a.transaction_datetime,
        a.transaction_date,
        a.purchase_id,
        a.buyer_id,
        a.prod_item_id,
        a.order_date,
        a.release_date,
        a.producer_id,
        b.product_id,
        b.item_quantity,
        b.purchase_value,
        c.subsidiary,
        current_timestamp() AS snapshot_datetime,
        DATE(current_timestamp()) AS snapshot_date
    FROM purchase_silver a
    LEFT JOIN purchase_extra_info_silver b on a.purchase_id = b.purchase_id and b.is_latest = true
    LEFT JOIN product_item_silver c on a.purchase_id = c.purchase_id and c.is_latest = true
    WHERE a.invoiced_status = 'Invoiced'
        AND a.is_latest = TRUE 
""")

In [38]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

window_snapshot = Window.partitionBy("purchase_id","transaction_date","snapshot_date") \
                        .orderBy(col("snapshot_datetime").desc())

df_gvm_union_dedup = (
    df_gvm_existing
    .unionByName(df_new_gvm)
    .withColumn("rn", row_number().over(window_snapshot))
    .filter(col("rn") == 1)
    .drop("rn")
)

In [39]:
df_gvm_union_dedup.show()

+--------------------+-----------+--------+------------+----------+------------+-----------+----------+-------------+--------------+-------------+--------------------+-------------+----------------+
|transaction_datetime|purchase_id|buyer_id|prod_item_id|order_date|release_date|producer_id|product_id|item_quantity|purchase_value|   subsidiary|   snapshot_datetime|snapshot_date|transaction_date|
+--------------------+-----------+--------+------------+----------+------------+-----------+----------+-------------+--------------+-------------+--------------------+-------------+----------------+
| 2023-07-15 09:00:00|         55|  160001|           5|2023-01-20|  2023-03-01|     852852|    696969|           10|         55.00|     nacional|2026-01-15 00:12:...|   2026-01-15|      2023-07-15|
| 2023-02-26 03:00:00|         69|  160001|          18|2023-01-26|  2023-02-28|      96967|    373737|            2|       2000.00|internacional|2026-01-15 00:12:...|   2026-01-15|      2023-02-26|
+----

In [40]:
# =====================================================
# 7. Escrita final da Silver
# =====================================================
df_gvm_gold.write \
    .format("parquet") \
    .mode("append") \
    .partitionBy("snapshot_date","transaction_date") \
    .save("data_lake/gold/gvm")